In [1]:
from keplergl import KeplerGl
from geopy.geocoders import Nominatim
import pandas as pd
import time
from geopy.exc import GeocoderTimedOut
import datetime
from decimal import Decimal
geolocator=Nominatim(timeout=10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


# Settings

## File to import

In [2]:
# Original file with station locations
pathStations = "./h2live_existing_FuelStations.txt"

# Old file with refueling sessions
importListOfRefueling = "./h2live_data_2019_10_23_00_00_56.txt"

# New file with all refueling sessions
importCompleteListOfRefueling = "./refuel_short_2.csv"

## Created files

In [3]:
# Path to export H2 stations with geo information
exportPathCSV = "./geoH2.csv"

# Path to import H2 stations with geo information
importGeoH2 = "./geoH2_v3.csv"

# Path to export complete file for Kepler
exportPathInputKepler = "./InputKepler.csv"

# Path to export complete file for Kepler
exportPathCompleteInputKepler = "./InputCompleteKepler.csv"

# Import H2 Stations

In [4]:
locationH2 = pd.read_csv(pathStations, sep= ";")
locationH2 = locationH2.loc[locationH2["country"] == "DE"]

In [5]:
locationH2.head()

,id,town,road,number,plz,country,operator
0,100,Aachen (Shell),Prager Ring,106,52070,DE,H2 MOBILITY
9,111,Bad Rappenau (Autohof 24),Buchäckerring,40,74906,DE,H2 MOBILITY
10,112,Bayreuth (Shell),Christian-Ritter-von-Langheinrich,2,95448,DE,H2 MOBILITY
12,114,Berg bei Hof (Shell),Sieggrubenstraße,5,95180,DE,H2 MOBILITY
13,116,Berlin - Heerstraße (TOTAL),Heerstraße,37,14055,DE,H2 MOBILITY


In [6]:
geoH2 = pd.DataFrame(columns=["id", "lat", "long", "city", "road", "number"])
geolocator = Nominatim(user_agent="H2 Test2")

for i in locationH2["id"].index:
    
    print(locationH2.at[i, "town"].split("(")[0].split("-")[0] + " " 
                              + locationH2.at[i, "road"] + " " 
                              + locationH2.at[i, "number"])
    
    # Try to get location, if TimedOut, try again
    try:
        location = geolocator.geocode(locationH2.at[i, "town"].split("(")[0].split("-")[0] + " " 
                              + locationH2.at[i, "road"] + " " 
                              + locationH2.at[i, "number"], 
                                 timeout = 20)
    except GeocoderTimedOut as e:
        print("Error: Second try")
        location = geolocator.geocode(locationH2.at[i, "town"].split("(")[0].split("-")[0] + " " 
                              + locationH2.at[i, "road"] + " " 
                              + locationH2.at[i, "number"], 
                                 timeout = 20)

    # Try accesing the lat/long. If not available, use 0/0
    try:
        geoH2 = geoH2.append({"id": locationH2.at[i, "id"], 
                              "lat": location.latitude, 
                              "long": location.longitude,  
                              "city": locationH2.at[i, "town"].split("(")[0].split("-")[0], 
                              "road": locationH2.at[i, "road"], 
                              "number": locationH2.at[i, "number"]}, ignore_index=True)
        print((location.latitude, location.longitude))
        
    except AttributeError as a: 
        geoH2 = geoH2.append({"id": locationH2.at[i, "id"],
                              "lat": 0, 
                              "long": 0,  
                              "city": locationH2.at[i, "town"].split("(")[0].split("-")[0], 
                              "road": locationH2.at[i, "road"], 
                              "number": locationH2.at[i, "number"]}, ignore_index=True)

    # Required by the API
    time.sleep(1)

# Export Results    
geoH2.to_csv(path_or_buf = exportPathCSV, index = None, sep= ";")

Aachen  Prager Ring 106
(50.7921029, 6.1185974)
Bad Rappenau  Buchäckerring 40
(49.2120935, 9.0769819)
Bayreuth  Christian-Ritter-von-Langheinrich 2
(49.96278135, 11.6107850192525)
Berg bei Hof  Sieggrubenstraße  5
(50.37194885, 11.7886343857023)
Berlin  Heerstraße 37
(52.4925883, 13.5730593)
Berlin  Heidestraße 19
(52.5346916, 13.3645584)
Berlin  Holzmarktstraße 36 - 42
(52.5114331, 13.4288729)
Berlin  Rothenbachstr.  1
(52.5716016, 13.4296019)
Berlin  Sachsendamm 90 - 92
(52.4786467, 13.3550598)
Bremen  Osterholzer Heerstraße 222
(53.0577831, 8.9625129)
Derching  Winterbruckenweg			 53
(48.4071187, 10.9513349204483)
Dresden  Wiener Strasse 39
(51.0365149, 13.7486437)
Duisburg  Am Schlütershof  27
(51.43819415, 6.73897718340605)
Düsseldorf  Höherweg 202
(51.2224356, 6.8203181)
Düsseldorf  Oerschbachstraße 71
(51.1893182, 6.8481428)
Essen  Katernberger Str.		 99
(51.5031007, 7.0490885)
Fellbach  Ohmstraße 25
(48.82230215, 9.28537070224665)
Frankfurt  Hanauer Landstr. 334
(50.1232501, 8

# Create timeline (old)

# Create Timeline (new)

In [7]:
listOfStations = pd.read_csv(importGeoH2, sep= ";")
completeListOfRefueling = pd.read_csv(importCompleteListOfRefueling, sep= ",")

inputKepler = pd.DataFrame(columns=["id", "time", "lat", "long", "city", "road", "number", "weight", "type"])

for index, row in completeListOfRefueling.iterrows():
    
    id = row["fk_Gas_Station_ID"]
    
    if id in listOfStations["id"].values:
        inputKepler = inputKepler.append({"id": id,
                                         "time": row["Timestamp"],
                                         "lat": listOfStations.loc[listOfStations['id'] == id].values[0][1],
                                         "long": listOfStations.loc[listOfStations['id'] == id].values[0][2],
                                         "city": listOfStations.loc[listOfStations['id'] == id].values[0][3], 
                                         "road": listOfStations.loc[listOfStations['id'] == id].values[0][4],
                                         "number": listOfStations.loc[listOfStations['id'] == id].values[0][5],
                                         "weight": row["Amount"], 
                                         "type": row["fk_Status"]}, ignore_index=True)
        
inputKepler.to_csv(path_or_buf = exportPathCompleteInputKepler, index = None, sep= ";")

# Visualization

In [8]:
map_1 = KeplerGl(height=500)

h2_stations = pd.read_csv(exportPathCompleteInputKepler, sep=";")
map_1.add_data(data=h2_stations, name='H2 Stationen')

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [10]:
map_1

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [], 'interactionConfig': {'…

In [11]:
config = map_1.config

In [12]:
map_2_2weeks = KeplerGl(height=500, data={"H2 Stationen": h2_stations}, config=config)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [13]:
map_2_2weeks

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [], 'interactionConfig': {'…

In [16]:
map_2_2weeks.save_to_html(file_name='hydrogen_map_two_weeks_3.html')

Map saved to hydrogen_map_two_weeks_3.html!


# Geo Locator

In [17]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.geocode("Bremen Osterholzer Heerstraße 222", timeout = 10)

In [18]:
print((location.latitude, location.longitude))

(53.0577831, 8.9625129)
